In [ ]:
!pip install plotly
!pip install dash
!pip install jupyter_dash
!pip install dash_bootstrap_components
!pip install dash_bootstrap_templates
!pip install emoji

In [ ]:
!pip install pythainlp

In [ ]:
import csv
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import numpy as np
import pandas as pd
import string
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import normalize

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
path = '/content/pollution_20230807.xlsx'

In [ ]:
data = pd.read_excel(path)

In [ ]:
data

,Unnamed: 0,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,lon,lat
0,23,2021-BX7JYW,{ถนน},NaN,ถนนเจริญนคร บริเวณแยกบุคคโล มีรถบรรทุกออกจากที...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.48923,13.70383",ตลาดจักรกล แขวง สำเหร่ เขตธนบุรี กรุงเทพมหานคร...,สำเหร่,ธนบุรี,กรุงเทพมหานคร,2021-12-09 11:51:30.682666+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:29:06.410309+00,100.48923,13.70383
1,109,2021-4QKUUN,{จราจร},NaN,ลดฝุ่น PM จากรถควันดำ\n1.ลดจำนวนรถเมล์ขสมก.ควั...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.55281,13.74219",4 ถ. สุขุมวิท แขวง คลองเตย เขตคลองเตย กรุงเทพม...,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,2021-12-18 07:36:06.837338+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:24:49.18165+00,100.55281,13.74219
2,139,2021-H4RRY3,{จราจร},NaN,โซนแยกบางพลัด รถติดมาก เพราะตำรวจเข้ามาวุ่นวาย...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.49413,13.78444",714 ถนน จรัญสนิทวงศ์ ถนนหน้าบ้าน แขวง บางยี่ขั...,บางยี่ขัน,บางพลัด,กรุงเทพมหานคร,2021-12-22 01:24:57.223484+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:23:18.714312+00,100.49413,13.78444
3,163,2021-6BBF82,{ความสะอาด},NaN,มีชาวบ้านแอบเผาขยะริมทางด่วน,https://storage.googleapis.com/traffy_public_b...,NaN,"100.42716,13.62669",14 เทียนทะเล 20 แยก 6/2 แขวง แสมดำ เขตบางขุนเท...,แสมดำ,บางขุนเทียน,กรุงเทพมหานคร,2021-12-23 12:07:19.556566+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:22:06.558111+00,100.42716,13.62669
4,180,2021-BVB8FX,{},NaN,กลุ่มควัน,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53599,13.76033",31 Phyathai Building 2nd Floor Room 211 Phyath...,ถนนพญาไท,ราชเทวี,กรุงเทพมหานคร,2021-12-29 12:28:59.692951+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:21:13.967309+00,100.53599,13.76033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7740,343784,2023-7BPZQH,"{ถนน,ร้องเรียน,PM2.5}","เขตห้วยขวาง,สำนักสิ่งแวดล้อม กทม.,ฝ่ายเทศกิจ เ...",แจ้งปัญหาไปหลายรอบ แต่เจ้าหน้าที่กลับไม่สามารถ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.57825,13.78393",26 ซอย รัชดาภิเษก 18 แขวงห้วยขวาง เขตห้วยขวาง ...,ห้วยขวาง,ห้วยขวาง,กรุงเทพมหานคร,2023-08-07 01:12:05.277932+00,กำลังดำเนินการ,NaN,0.0,2023-08-07 02:13:53.290396+00,100.57825,13.78393
7741,343786,2023-CVWVCC,{},"เขตบางแค,สำนักสิ่งแวดล้อม กทม.,ฝ่ายสิ่งแวดล้อม...",ควันจากการปิ้งย่าง,https://storage.googleapis.com/traffy_public_b...,NaN,"100.39191,13.68892",1095/108 บ้านพฤกษา 84/2 ซอย เพชรเกษม 63 แขวง ห...,หลักสอง,บางแค,กรุงเทพมหานคร,2023-08-07 01:12:41.808284+00,กำลังดำเนินการ,NaN,0.0,2023-08-07 02:03:03.691302+00,100.39191,13.68892
7742,343810,2023-EM8UL8,"{ความปลอดภัย,PM2.5}","เขตจตุจักร,สำนักสิ่งแวดล้อม กทม.,ฝ่ายสิ่งแวดล้...",มีการขายอาหารปิ้ง สร้างควัน pm2.5 จำนวนมาก สร้...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.54199,13.83297",13-26 S. Cement Thai แขวงลาดยาว เขตจตุจักร กรุ...,ลาดยาว,จตุจักร,กรุงเทพมหานคร,2023-08-07 01:41:42.102343+00,กำลังดำเนินการ,NaN,0.0,2023-08-07 01:44:26.104487+00,100.54199,13.83297
7743,343821,PKMNHH,"{ร้องเรียน,PM2.5,ถนน,ความปลอดภัย}","เขตพระโขนง,ฝ่ายโยธา เขตพระโขนง",ปัญหา: เดือดร้อนรำคาญจากฝุ่นและเสียง ผู้แจ้งอ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.61726,13.69394",31/1 ซอย ปุณณวิถี 29 แขวงบางจาก เขตพระโขนง กรุ...,บางจาก,พระโขนง,กรุงเทพมหานคร,2023-08-07 01:57:31.813261+00,กำลังดำเนินการ,NaN,0.0,2023-08-07 02:16:47.8266+00,100.61726,13.69394


In [ ]:
def process_text(text):

    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = list(thai_stopwords())
    list_word_not_stopwords = [i for i in tokens if i not in stop_words and i!=' ']

    return list_word_not_stopwords

In [ ]:
def gram_count(file_name, n_gram):
    # Read the file into a DataFrame
    df = pd.read_excel(path)

    # Extract the text data and preprocess it
    text_data = data['comment'].tolist()
    processed_text = [process_text(str(text)) for text in text_data]

    # Combine the preprocessed text into a single string
    # processed_text = ' '.join(processed_text)

    # Tokenize the text into words and remove stopwords
    stop_words = list(thai_stopwords())
    words = [word for word in processed_text if word not in stop_words and word!="\n"]
    bow = []
    for sent in words:
      for s in sent:
        bow.append(s)
    # Generate the n-grams and count their frequencies
    n_grams = ngrams(bow,n_gram)
    n_gram_counts = Counter(n_grams)

    return n_gram_counts

In [ ]:
test_txt = data.iloc[90]['comment']

In [ ]:
def sort_n_gram(n_gram):
    df = pd.DataFrame.from_dict(n_gram, orient='index', columns=['Frequency'])

    # Sort the dataframe by frequency in descending order
    df = df.sort_values(by=['Frequency'], ascending=False).reset_index()
    df.columns = ['word','count']
    df['word'] = df['word'].apply(lambda x: re.sub('[^ก-๙]',' ', str(x)))
    df['word'] = df['word'].apply(lambda x: re.sub('\s+',' ', str(x)))
    df = df.query('word!=" "')
    return df

## Overview

### 1-gram

In [ ]:
one_gram = sort_n_gram(gram_count(path,1))

In [ ]:
import plotly.express as px
px.bar(
    one_gram[:20],
    y = 'word',
    x = 'count'
).update_layout( title="Overview Bangkok 1-gram",yaxis={'categoryorder':'total ascending'})

### 2-gram

In [ ]:
two_gram = gram_count(path,2)

In [ ]:
df = pd.DataFrame.from_dict(two_gram, orient='index', columns=['Frequency'])

    # Sort the dataframe by frequency in descending order
df = df.sort_values(by=['Frequency'], ascending=False).reset_index()
df.columns = ['word','count']
df['word'] = df['word'].apply(lambda x: re.sub('[^ก-๙]',' ', str(x)))
df['word'] = df['word'].apply(lambda x: re.sub('\s+',' ', str(x)))
two_gram = df.query('word!=" "')

In [ ]:
px.bar(
    df[:20],
    y = 'word',
    x = 'count'
).update_layout( title="Overview Bangkok 2-gram",yaxis={'categoryorder':'total ascending'})

### 3-gram

In [ ]:
three_gram = gram_count(path,3)

In [ ]:
df = pd.DataFrame.from_dict(three_gram, orient='index', columns=['Frequency'])

    # Sort the dataframe by frequency in descending order
df = df.sort_values(by=['Frequency'], ascending=False).reset_index()
df.columns = ['word','count']
df['word'] = df['word'].apply(lambda x: re.sub('[^ก-๙]',' ', str(x)))
df['word'] = df['word'].apply(lambda x: re.sub('\s+',' ', str(x)))
df = df.query('word!=" "')

In [ ]:
df['length'] = df['word'].apply(lambda x: len(x.split()))
df = df[df['length']==3]

In [ ]:
px.bar(
    df[:20],
    y = 'word',
    x = 'count'
).update_layout( title="Overview Bangkok 3-gram",yaxis={'categoryorder':'total ascending'})

## By District

In [ ]:
df_all = pd.read_excel(path)

In [ ]:
def gram_count_by_district(n_gram, district):
    # Read the file into a DataFrame
    data = df_all[df_all['district']==district]
    # Extract the text data and preprocess it
    text_data = data['comment'].tolist()
    processed_text = [process_text(str(text)) for text in text_data]

    # Combine the preprocessed text into a single string
    # processed_text = ' '.join(processed_text)

    # Tokenize the text into words and remove stopwords
    stop_words = list(thai_stopwords())
    words = [word for word in processed_text if word not in stop_words and word!="\n"]
    bow = []
    for sent in words:
      for s in sent:
        bow.append(s)
    # Generate the n-grams and count their frequencies
    n_grams = ngrams(bow,n_gram)
    n_gram_counts = Counter(n_grams)

    return n_gram_counts

def data_by_district(n,district):
  one = gram_count_by_district(n,district)
  df = pd.DataFrame.from_dict(one, orient='index', columns=['Frequency'])
  keyword = ['ธนบุรี','บ้าน','ซอย','เขต','ถนน',"เจ้าของ","เลขที่","วัด","ท่า",'ฉัน',"แจ้ง","เรื่อง","บริเวณ","ปัญหา"]
      # Sort the dataframe by frequency in descending order
  df = df.sort_values(by=['Frequency'], ascending=False).reset_index()
  df.columns = ['word','count']
  df['word'] = df['word'].apply(lambda x: re.sub('[^ก-๙]',' ', str(x)))
  # df['word'] = df['word'].apply(lambda x: re.sub(f'[^"{district}"]',' ', str(x)))
  df['word'] = df['word'].apply(lambda x: re.sub('\s+',' ', str(x)))
  df = df.query('word!=" "')
  df['length'] = df['word'].apply(lambda x: len(x.split()))
  df = df[df['length']==n]
  df = df[~df['word'].str.contains('|'.join(keyword)).any(level=0)]

  return df

In [ ]:
districts = ["คลองเตย","คลองสาน","คลองสามวา","คันนายาว","จตุจักร","จอมทอง"
            ,"ดอนเมือง","ดินแดง","ดุสิต","ตลิ่งชัน","ทวีวัฒนา","ทุ่งครุ","ธนบุรี"
            ,"บางเขน","บางแค","บางกอกใหญ่","บางกอกน้อย","บางกะปิ","บางขุนเทียน"
            ,"บางคอแหลม","บางซื่อ","บางนา","บางบอน","บางพลัด","บางรัก","บึงกุ่ม"
            ,"ปทุมวัน","ประเวศ","ป้อมปราบศัตรูพ่าย","พญาไท","พระโขนง","พระนคร"
            ,"ภาษีเจริญ","มีนบุรี","ยานนาวา","ราชเทวี","ราษฎร์บูรณะ","ลาดกระบัง"
            ,"ลาดพร้าว","วังทองหลาง","วัฒนา","สวนหลวง","สะพานสูง","สัมพันธวงศ์"
            ,"สาทร","สายไหม","หนองแขม","หนองจอก","หลักสี่","ห้วยขวาง"]
data_df = {}
for i in range (1,3):
  for district in districts:
    df = data_by_district(i,district)
    if district not in data_df.keys():
      data_df[district] = {i:df[:20]}
    else:
        data_df[district][i] = df[:20]
data_df['ภาพรวมกรุงเทพมหานคร'] = {1:one_gram[:20],2:two_gram[:20]}

<ipython-input-22-672d2ea0319e>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-22-672d2ea0319e>:14: FutureWarning:

Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.any(level=1) should use df.groupby(level=1).any()

<ipython-input-22-672d2ea0319e>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-22-672d2ea0319e>:14: FutureWarning:

Using the level keyword in DataFrame and Series aggre

In [ ]:
data_df['ภาพรวมกรุงเทพมหานคร']

{1:           word  count
 1        ฝุ่น    3291
 2         เผา    2988
 3        ควัน    2962
 5        บ้าน    2503
 6         ถนน    2438
 7       กลิ่น    2429
 8        แจ้ง    2354
 9         ซอย    2310
 10        ขยะ    2173
 11         รถ    2129
 12     บริเวณ    1912
 14        เขต    1774
 15     เรื่อง    1759
 16      รบกวน    1634
 18         คน    1491
 19   ก่อสร้าง    1458
 21         ทำ    1402
 22      ปัญหา    1391
 23       ร้าน    1330
 24      เหม็น    1276,
 2:                      word  count
 1                เผา ขยะ     949
 2                ควัน ดำ     431
 3            กลิ่น เหม็น     411
 4             กลิ่น ควัน     411
 5            แจ้ง เรื่อง     372
 6         ส่งกลิ่น เหม็น     332
 10   สร้าง ความเดือดร้อน     262
 12      บริเวณ ใกล้เคียง     227
 15           บ้าน เลขที่     212
 17              เผา หญ้า     193
 18             ฝุ่น ควัน     190
 21          มลภาวะ เสียง     187
 22            ปล่อย ควัน     187
 23                จอด รถ     180


In [ ]:
import json
# Opening JSON file
f = open('/content/WordGramCount.json')

# returns JSON object as
# a dictionary
data_json = json.load(f)


# Closing file
f.close()